In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 맷플롯립 한글 사용 가능
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

# 선택
from scipy.spatial import cKDTree

In [14]:
# 파일 불러오기
apt_df = pd.read_csv('Data_Preprocessing/apt_information_with_l_and_l.csv')
park_df = pd.read_csv('../6.Infrastructure/공원좌표.csv', encoding='cp949')
bus_df = pd.read_csv('../4.Public_transportation/Bus_stop_latitude_and_longitude.csv', encoding='cp949')
station_df = pd.read_csv('../4.Public_transportation/subway_latitude_and_longitude.csv', encoding='cp949')
school_df = pd.read_csv('forDP/from_Jey_uniq_all_school.csv')
hospital_df = pd.read_csv('forDP/from_Jey_uniq_hospital.csv')

In [15]:
# 기본 정보 확인
apt_df.info(), park_df.info(), bus_df.info(), station_df.info(), school_df.info(), hospital_df.info()
apt_df.columns, park_df.columns, bus_df.columns, station_df.columns, school_df.columns, hospital_df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251096 entries, 0 to 251095
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   NO                 251096 non-null  object 
 1   contract_date      251096 non-null  object 
 2   apt_name           251096 non-null  object 
 3   exclusive_area_m2  251096 non-null  float64
 4   price_man_won      251096 non-null  int64  
 5   floor              251096 non-null  int64  
 6   construction_year  251096 non-null  int64  
 7   latitude           249113 non-null  float64
 8   longitude          249113 non-null  float64
 9   adjusted_price     251096 non-null  float64
dtypes: float64(4), int64(3), object(3)
memory usage: 19.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   공원명           130 non-null    object 
 1   

(Index(['NO', 'contract_date', 'apt_name', 'exclusive_area_m2', 'price_man_won',
        'floor', 'construction_year', 'latitude', 'longitude',
        'adjusted_price'],
       dtype='object'),
 Index(['공원명', 'X좌표(GRS80TM)', 'Y좌표(GRS80TM)', 'X좌표(WGS84)', 'Y좌표(WGS84)'], dtype='object'),
 Index(['ID', 'bus_stop_name', 'latitude', 'longitude'], dtype='object'),
 Index(['ID', 'name', 'line', 'latitude', 'longitude'], dtype='object'),
 Index(['no', 'address', 'latitude', 'longitude'], dtype='object'),
 Index(['no', 'address', 'latitude', 'longitude'], dtype='object'))

In [16]:
apt_df.isna().sum(), park_df.isna().sum(), bus_df.isna().sum(), station_df.isna().sum(), school_df.isna().sum(), hospital_df.isna().sum()

(NO                      0
 contract_date           0
 apt_name                0
 exclusive_area_m2       0
 price_man_won           0
 floor                   0
 construction_year       0
 latitude             1983
 longitude            1983
 adjusted_price          0
 dtype: int64,
 공원명              0
 X좌표(GRS80TM)    11
 Y좌표(GRS80TM)    11
 X좌표(WGS84)      11
 Y좌표(WGS84)      11
 dtype: int64,
 ID               0
 bus_stop_name    0
 latitude         0
 longitude        0
 dtype: int64,
 ID           0
 name         0
 line         0
 latitude     0
 longitude    0
 dtype: int64,
 no            0
 address       0
 latitude     19
 longitude    19
 dtype: int64,
 no              0
 address         0
 latitude     1155
 longitude    1155
 dtype: int64)

In [6]:
# 결측치 처리
apt_df.isna().sum(), park_df.isna().sum(), bus_df.isna().sum(), station_df.isna().sum(), school_df.isna().sum(), hospital_df.isna().sum()
apt_df = apt_df.dropna()
park_df = park_df.dropna()
school_df = school_df.dropna()
hospital_df = hospital_df.dropna()
apt_df.isna().sum(), park_df.isna().sum(), bus_df.isna().sum(), station_df.isna().sum(), school_df.isna().sum(), hospital_df.isna().sum()

(NO                   0
 contract_date        0
 apt_name             0
 exclusive_area_m2    0
 price_man_won        0
 floor                0
 construction_year    0
 latitude             0
 longitude            0
 adjusted_price       0
 dtype: int64,
 공원명             0
 X좌표(GRS80TM)    0
 Y좌표(GRS80TM)    0
 X좌표(WGS84)      0
 Y좌표(WGS84)      0
 dtype: int64,
 ID               0
 bus_stop_name    0
 latitude         0
 longitude        0
 dtype: int64,
 ID           0
 name         0
 line         0
 latitude     0
 longitude    0
 dtype: int64,
 no           0
 address      0
 latitude     0
 longitude    0
 dtype: int64,
 no           0
 address      0
 latitude     0
 longitude    0
 dtype: int64)

In [7]:
# 컬럼 정리
apt_df = apt_df[['NO', 'contract_date', 'latitude', 'longitude']]

park_df.columns = ['park_name', 'x', 'y', 'longitude', 'latitude']
park_df = park_df[['park_name', 'latitude', 'longitude']]

bus_df.columns = ['bus_id', 'bus_stop_name', 'latitude', 'longitude']

station_df['line_name'] = station_df['line'] + '_' + station_df['name']
station_df = station_df.drop_duplicates(subset='line_name')
station_df.columns = ['station_id', 'station_name', 'line', 'latitude', 'longitude', 'line_name']
station_df = station_df[['station_id', 'line_name', 'latitude', 'longitude']]

school_df = school_df[['no', 'latitude', 'longitude']]
school_df.columns = ['school_name', 'latitude', 'longitude']

hospital_df = hospital_df[['no', 'latitude', 'longitude']]
hospital_df.columns = ['hospital_name', 'latitude', 'longitude']

In [8]:
apt_df.describe(), station_df.describe(), bus_df.describe(), park_df.describe(), school_df.describe(), hospital_df.describe()

(            latitude      longitude
 count  249113.000000  249113.000000
 mean       37.555326     126.995731
 std         0.056523       0.090485
 min        37.434420     126.806400
 25%        37.507361     126.917908
 50%        37.549681     127.017803
 75%        37.599797     127.065753
 max        37.687503     127.179998,
         station_id    latitude   longitude
 count   782.000000  782.000000  782.000000
 mean   2389.792839   37.511847  126.971378
 std    1593.851659    0.154688    0.182881
 min     150.000000   36.769502  126.441442
 25%    1285.250000   37.470522  126.853044
 50%    2511.500000   37.521754  126.997706
 75%    3137.750000   37.577946  127.073403
 max    9996.000000   38.100730  127.723792,
              bus_id      latitude     longitude
 count  1.129000e+04  11290.000000  11290.000000
 mean   1.132294e+08     37.550071    126.985975
 std    6.993850e+06      0.054553      0.086257
 min    1.000000e+08     37.430537    126.721031
 25%    1.079001e+08    

In [9]:
def calculate_nearest_facilities(apt_df, ref_df, ref_name, radius_km_list=[0.5, 1, 1.5], name_column='name'):
    """
    아파트 기준으로 가장 가까운 시설 이름 및 거리 계산 함수.
    apt_df: 아파트 데이터프레임
    ref_df: 기준 데이터프레임 (공원, 버스정류장, 지하철역 등)
    ref_name: 기준 이름 ('공원', '버스정류장', '지하철역' 등)
    radius_km_list: 계산할 반경 (km 단위)
    name_column: 시설 이름 컬럼 (기본값 'name')
    """
    # 결과 저장용 딕셔너리
    results = {f'{ref_name}_within_{r}km': [] for r in radius_km_list}
    results[f'closest_{ref_name}_dist_km'] = []
    results[f'closest_{ref_name}_name'] = []

    # KDTree 생성 (ref_df 기준 좌표를 라디안으로 변환)
    ref_tree = cKDTree(np.radians(ref_df[['latitude', 'longitude']].values))

    for _, apt_row in apt_df.iterrows():
        apt_coords = np.radians([apt_row['latitude'], apt_row['longitude']])  # 아파트 좌표 라디안 변환
        
        # 반경 내 개수 계산
        for radius_km in radius_km_list:
            radius_rad = radius_km / 6371.0  # 반경을 라디안 단위로 변환
            count = len(ref_tree.query_ball_point(apt_coords, r=radius_rad))
            results[f'{ref_name}_within_{radius_km}km'].append(count)
        
        # 가장 가까운 거리 및 인덱스 계산
        dist, idx = ref_tree.query(apt_coords, k=1)
        closest_distance = dist * 6371.0  # 라디안 -> km 변환
        closest_name = ref_df.iloc[idx][name_column]  # 인덱스를 통해 시설 이름 가져오기
        
        results[f'closest_{ref_name}_dist_km'].append(closest_distance)
        results[f'closest_{ref_name}_name'].append(closest_name)

    # 결과를 apt_df에 추가
    for col_name, col_values in results.items():
        apt_df[col_name] = col_values

    return apt_df


In [10]:
# 지하철역
apt_df = calculate_nearest_facilities(apt_df, station_df, 'station', name_column='line_name')

# 버스정류장
apt_df = calculate_nearest_facilities(apt_df, bus_df, 'bus_stop', name_column='bus_stop_name')

# 공원
apt_df = calculate_nearest_facilities(apt_df, park_df, 'park', name_column='park_name')

# 병원
apt_df = calculate_nearest_facilities(apt_df, hospital_df, 'hospital', name_column='hospital_name')

# 학교
apt_df = calculate_nearest_facilities(apt_df, school_df, 'school', name_column='school_name')


In [11]:
# 새로 추가된 데이터를 확인
apt_df.info()
print(apt_df.head())

<class 'pandas.core.frame.DataFrame'>
Index: 249113 entries, 0 to 251095
Data columns (total 29 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   NO                        249113 non-null  object 
 1   contract_date             249113 non-null  object 
 2   latitude                  249113 non-null  float64
 3   longitude                 249113 non-null  float64
 4   station_within_0.5km      249113 non-null  int64  
 5   station_within_1km        249113 non-null  int64  
 6   station_within_1.5km      249113 non-null  int64  
 7   closest_station_dist_km   249113 non-null  float64
 8   closest_station_name      249113 non-null  object 
 9   bus_stop_within_0.5km     249113 non-null  int64  
 10  bus_stop_within_1km       249113 non-null  int64  
 11  bus_stop_within_1.5km     249113 non-null  int64  
 12  closest_bus_stop_dist_km  249113 non-null  float64
 13  closest_bus_stop_name     249113 non-null  object

In [12]:
apt_df[apt_df['closest_station_dist_km']>3]

,NO,contract_date,latitude,longitude,station_within_0.5km,station_within_1km,station_within_1.5km,closest_station_dist_km,closest_station_name,bus_stop_within_0.5km,...,hospital_within_0.5km,hospital_within_1km,hospital_within_1.5km,closest_hospital_dist_km,closest_hospital_name,school_within_0.5km,school_within_1km,school_within_1.5km,closest_school_dist_km,closest_school_name
1181,2019_01182,2019-12-27,37.609704,126.975875,0,0,0,3.609688,우이신설선_북한산보국문,6,...,2,6,11,0.090044,하루학문외과의원,0,1,1,0.990368,서울예술고등학교
2105,2019_02106,2019-12-23,37.609704,126.975875,0,0,0,3.609688,우이신설선_북한산보국문,6,...,2,6,11,0.090044,하루학문외과의원,0,1,1,0.990368,서울예술고등학교
3203,2019_03204,2019-12-18,37.608512,126.974317,0,0,0,3.642644,3호선_경복궁(정부서울청사),4,...,1,8,11,0.190095,하루학문외과의원,0,1,1,0.777415,서울예술고등학교
4379,2019_04380,2019-12-14,37.610799,126.978411,0,0,0,3.321021,우이신설선_북한산보국문,7,...,2,3,9,0.274132,코숨한의원,0,0,1,1.297275,서울예술고등학교
10006,2019_10007,2019-11-30,37.608882,126.975423,0,0,0,3.667545,우이신설선_북한산보국문,5,...,1,8,11,0.130321,하루학문외과의원,0,1,1,0.906751,서울예술고등학교
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230152,2023_14699,2023-07-24,37.608987,126.972990,0,0,0,3.694927,3호선_경복궁(정부서울청사),6,...,1,9,12,0.263870,하루학문외과의원,0,1,2,0.671163,서울예술고등학교
231908,2023_16455,2023-07-10,37.608260,126.973290,0,0,0,3.613695,3호선_경복궁(정부서울청사),6,...,1,9,12,0.282691,하루학문외과의원,0,1,2,0.661739,서울예술고등학교
231944,2023_16491,2023-07-09,37.610799,126.978411,0,0,0,3.321021,우이신설선_북한산보국문,7,...,2,3,9,0.274132,코숨한의원,0,0,1,1.297275,서울예술고등학교
237031,2023_21578,2023-05-31,37.608512,126.974317,0,0,0,3.642644,3호선_경복궁(정부서울청사),4,...,1,8,11,0.190095,하루학문외과의원,0,1,1,0.777415,서울예술고등학교


In [ ]:
station_df.head()

In [ ]:
station_df.describe()

In [ ]:
bus_df.describe()

In [ ]:
apt_df.describe()

In [ ]:
park_df.describe()